本ノートブックでは、用意したデータをまとめて扱いやすい形式にします。  
<!-- 気温データについて、時系列手法を試してみましょう。   -->
このデータセットは気象庁のホームページ (https://www.data.jma.go.jp/gmd/risk/obsdl/index.php) から取得した、データです。
地点は
<ul>
    <li>京都市</li>
</ul>
で、項目は
<ul>
    <li>1時間おきの気温</li>
    <li>1時間おきの日照時間（前1時間）</li>
</ul>
で、期間は
<ul>
    <li>2001年1月1日-2021年6月30日</li>
</ul>
です。  
データは ./dataset_kyoto_temp/ に入れています。  

参考:  


<!-- https://www.dezyre.com/recipes/run-basic-rnn-model-using-pytorch -->

In [1]:
from glob import glob
import pandas as pd
# import torch
# import torch.nn as nn


# データ

## データセットの確認

まずはデータセットを確認してみましょう。

In [2]:
sorted(glob("./data/*")) # データセットの一覧

['./data/temp_kyoto2000.csv',
 './data/temp_kyoto2001.csv',
 './data/temp_kyoto2002.csv',
 './data/temp_kyoto2003.csv',
 './data/temp_kyoto2004.csv',
 './data/temp_kyoto2005.csv',
 './data/temp_kyoto2006.csv',
 './data/temp_kyoto2007.csv',
 './data/temp_kyoto2008.csv',
 './data/temp_kyoto2009.csv',
 './data/temp_kyoto2010.csv',
 './data/temp_kyoto2011.csv',
 './data/temp_kyoto2012.csv',
 './data/temp_kyoto2013.csv',
 './data/temp_kyoto2014.csv',
 './data/temp_kyoto2015.csv',
 './data/temp_kyoto2016.csv',
 './data/temp_kyoto2017.csv',
 './data/temp_kyoto2018.csv',
 './data/temp_kyoto2019.csv']

1年分が一つのcsvファイルに入っています。  

## データセットの読み込み

以上2000年-2019年のデータセットをそれぞれ読み込み、一つのデータフレームにまとめます。

In [6]:
D=pd.DataFrame() # まとめたデータフレーム

for y in range(2000,2020): # 各年において
    fname="./data/temp_kyoto{}.csv".format(y) # ファイル名
    df=pd.read_csv(fname,encoding="SHIFT_JIS",header=3) # データフレームに格納
    df[["日付","時間"]]=df["Unnamed: 0"].str.split(expand=True) # 日時を日付と時間に分割
    df.rename({"Unnamed: 1":"気温"},inplace=True,axis=1) # 気温の項目の名前づけ
    df.drop(columns=["Unnamed: 0",u"品質情報",u"均質番号"],axis=1,inplace=True) # 不要な項目は削除
    
    D=pd.concat([D,df],axis=0) # 一つのデータフレームにまとめる
    
D.reset_index(drop=True,inplace=True) # データフレームのインデックスを整理する

print("D.shape = ",D.shape)
D.head()

D.shape =  (175320, 3)


,気温,日付,時間
0,5.2,2000/1/1,1:00:00
1,5.5,2000/1/1,2:00:00
2,5.8,2000/1/1,3:00:00
3,5.9,2000/1/1,4:00:00
4,5.7,2000/1/1,5:00:00


## データセットの分割

データを学習用の2000年-2017年、検証用の2018年とテスト用の2019年に分割します。

In [10]:
Dtr=D[D["日付"].str[:4]<="2017"] # 学習用: 2000年-2017年
Dva=D[D["日付"].str[:4]=="2018"] # 検証用: 2018年
Dte=D[D["日付"].str[:4]=="2019"] # テスト用: 2019年

print("Dtr.shape = ",Dtr.shape)
print("Dva.shape = ",Dva.shape)
print("Dte.shape = ",Dte.shape)

Dtr.shape =  (157799, 3)
Dva.shape =  (8760, 3)
Dte.shape =  (8760, 3)


# 学習の準備

## デバイスの準備

In [65]:
is_cuda=torch.cuda.is_available() # GPUが利用可能かどうか

if is_cuda: # GPUを利用できる場合
    device=torch.device("cuda")
    print("GPU is available")
else: # GPUを利用できない場合
    device=torch.device("cpu")
    print("GPU is not available, CPU used")

GPU is not available, CPU used


## DataLoaderの準備

まずはDataLoaderの準備をします。

In [94]:
batch_size=2 # バッチサイズ、2以上がよい

### 学習用データセットのローダ
loader_train=torch.utils.data.DataLoader(
    dataset=torch.tensor(Dtr["気温"].values).view(-1,1,1),
    batch_size=batch_size,
    shuffle=False
)

### 検証用データセットのローダ
loader_valid=torch.utils.data.DataLoader(
    dataset=torch.tensor(Dva["気温"].values).view(-1,1,1),
    batch_size=batch_size,
    shuffle=False
)

### テスト用データセットのローダ
loader_test=torch.utils.data.DataLoader(
    dataset=torch.tensor(Dte["気温"].values).view(-1,1,1),
    batch_size=batch_size,
    shuffle=False
)

## ネットワークの準備

ネットワークを定義します。pytorchのLSTMを使います。パラメータを設定します。

In [77]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM,self).__init__()
        
        # LSTM
        self.lstm=nn.LSTM(
            input_size=input_dim,
            hidden_size=10, # 隠れ層のユニットの数
            num_layers=1, # RNNの層の数
            batch_first=True # 入力と出力のテンソルの第1次元はバッチサイズとなる
        )
        self.out=nn.Linear(10,1) # 出力層
    
    def forward(self,x):
        # x.shape = (batch, time_step, input_size)
        # r_out.shape = (batch, time_step, output_size)
        # h_n.shape = (n_layers, batch, hidden_size)
        # h_c.shape = (n_layers, batch, hidden_size)
        r_out,(h_n,h_c)=self.lstm(x,None) # Noneは内部状態を0で初期化するという意味
        
        # 最後のステップのr_outのみ出力
        out=self.out(r_out[:,-1,:])
        
        return out

モデルを用意します。

In [78]:
lstm=LSTM() # ネットワーク
lstm.to(device) # モデルをデバイスに送る
print(lstm)

LSTM(
  (lstm): LSTM(1, 10, batch_first=True)
  (out): Linear(in_features=10, out_features=1, bias=True)
)


## 学習の準備

最適化の方法を設定します。

In [79]:
epoch=1 # エポック数
loss_func=nn.MSELoss() # ロス関数はRMSE（Rootを計算するのはもうちょい後）
lr=1e-5 # 学習率
optimizer=torch.optim.Adam(lstm.parameters(),lr=lr) # 最適化関数

# 学習

それでは学習させます。

In [86]:
for ep in range(epoch):
    ### 各エポックで学習させる
    for step,x in enumerate(loader_train):
        if step==1:
            break
        
        output=lstm(x) # ネットワークの出力
        
        loss=torch.sqrt(loss_func(output.unsqueeze(2),x)) # ロスの計算
        
        optimizer.zero_grad() # 勾配の初期化
        loss.backward() # 逆伝搬させる
        optimizer.step() # 最適化する
        
    
    ### 各エポックの最後に、検証用データでロスを算出する
    lstm.eval() # モデルを評価モードに

    for inp,lab in loader_valid:
        